In [316]:
# import packages to prep data with
import pandas as pd
import datetime
import re

## Reading in the datasets
It's always a good idea to print out the summary and first few rows of data so you can get an idea of what you're working with

In [317]:
title_data = pd.read_csv('title_akas.csv')
print(title_data.info())
title_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571 entries, 0 to 1570
Data columns (total 9 columns):
Unnamed: 0         1571 non-null int64
titleId            1571 non-null object
ordering           1571 non-null int64
title              1571 non-null object
region             1571 non-null object
language           38 non-null object
types              1041 non-null object
attributes         185 non-null object
isOriginalTitle    1571 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 110.5+ KB
None


,Unnamed: 0,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,80622,tt0160127,33,Charlie's Angels,US,NaN,NaN,NaN,0.0
1,80623,tt0160127,37,Charlie's Angels: The Movie,US,NaN,working,NaN,0.0
2,82565,tt0163579,16,Chill Factor,US,NaN,NaN,NaN,0.0
3,82587,tt0163651,10,East Great Falls High,US,NaN,working,NaN,0.0
4,82588,tt0163651,16,Great Falls,US,NaN,working,NaN,0.0


Looks like IDs follow the format ttXXXXXXX, I see some duplicates so I'll need to address those during the cleaning process. There's also columns I don't care about, so I'll edit those too.<br><br>
I'm going to read in the `movie_summary` table to get an idea of what that looks like

In [318]:
movie_summary = pd.read_csv('movie_summary.csv')
print(movie_summary.info())
movie_summary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
titleId            1001 non-null object
dom_total_sales    1001 non-null object
int_total_sales    1001 non-null object
dom_distributer    1000 non-null object
dom_opening        1001 non-null object
budget             1001 non-null object
release_date       1001 non-null object
mpaa_rating        1001 non-null object
dtypes: object(8)
memory usage: 62.6+ KB
None


,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating
0,tt0187393,"$113,330,342","$101,964,000",Sony Pictures ReleasingSee full company inform...,"$22,413,710","$110,000,000","June 28, 2000\n (Domestic)",R
1,tt0283897,"$1,013,272",~na~,United ArtistsSee full company information\n\n,"$64,474",~na~,R,1 hr 54 min
2,tt0475998,"$12,981,269","$266,456",Universal PicturesSee full company information...,"$4,251,320",~na~,G,1 hr 25 min
3,tt0204946,"$68,379,000","$22,070,929",Universal PicturesSee full company information...,"$17,362,105","$11,000,000","August 25, 2000\n (Domestic)",PG-13
4,tt1549920,"$12,050,299","$36,280,458",LionsgateSee full company information\n\n,"$6,281,433","$45,000,000","January 17, 2013\n (APAC, EMEA)",R


This looks pretty straightforward but there are some errors I'm going to have to clean up before sending this to the database.<br>
Ex. Some runtimes ended up in the `mpaa_rating` column, and some `mpaa_rating` values ended up in the `release_date` column
<br><br>
I'll address these later. For now, I'll look at the final table `dom_sales`

In [319]:
domestic_sales = pd.read_csv('dom_sales.csv')
print(domestic_sales.info())
domestic_sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64905 entries, 0 to 64904
Data columns (total 4 columns):
Date       64905 non-null object
Daily      64905 non-null object
Day        64905 non-null object
titleId    64905 non-null object
dtypes: object(4)
memory usage: 2.0+ MB
None


,Date,Daily,Day,titleId
0,"Jun 28, 2000","$5,015,267",1,tt0187393
1,"Jun 29, 2000","$4,307,385",2,tt0187393
2,"Jun 30, 2000","$6,860,000",3,tt0187393
3,"Jul 1, 2000","$8,560,000",4,tt0187393
4,"Jul 2, 2000","$6,990,000",5,tt0187393


As you can see, this table is the largest of the three. It has a many-to-one format.<br>
I'd say it's the cleanest out of all three but I can still make improvements on it.
<br><br><br>
## Cleaning the data
I'm returning to the first table to address some issues I saw earlier. I'm going to print a few more rows this time.

In [320]:
print(title_data.info())
title_data.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571 entries, 0 to 1570
Data columns (total 9 columns):
Unnamed: 0         1571 non-null int64
titleId            1571 non-null object
ordering           1571 non-null int64
title              1571 non-null object
region             1571 non-null object
language           38 non-null object
types              1041 non-null object
attributes         185 non-null object
isOriginalTitle    1571 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 110.5+ KB
None


,Unnamed: 0,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,80622,tt0160127,33,Charlie's Angels,US,NaN,NaN,NaN,0.0
1,80623,tt0160127,37,Charlie's Angels: The Movie,US,NaN,working,NaN,0.0
2,82565,tt0163579,16,Chill Factor,US,NaN,NaN,NaN,0.0
3,82587,tt0163651,10,East Great Falls High,US,NaN,working,NaN,0.0
4,82588,tt0163651,16,Great Falls,US,NaN,working,NaN,0.0
5,82589,tt0163651,1,American Pie,US,NaN,NaN,NaN,0.0
6,82590,tt0163651,21,Untitled Teenage Sex Comedy That Can Be Made f...,US,NaN,working,NaN,0.0
7,82591,tt0163651,26,Untitled Teenage Sex Project,US,NaN,working,NaN,0.0
8,82702,tt0163988,34,Bringing Out the Dead,US,NaN,imdbDisplay,NaN,0.0
9,83376,tt0165929,12,Romeo Must Die,US,NaN,imdbDisplay,NaN,0.0


I see some glaring issues right away. There are multiple IDs when ideally there should be one and there are some junk columns.

I'm going to drop the columns I don't need. I've decided to keep `titleId` and `title` for obvious reasons. There are only US movies because I filtered out other regions prior to this. <br>
The `language` column looks pretty useless so I'm going to toss that. Same deal with `attributes` and `isOriginalTitle`.
I'm going to keep `types` because it may help handle duplicate IDs. I was hoping that `ordering` would offer some insight with the duplicate IDs but I don't believe so, so I'll drop that too.<br>
Lastly, I'm going to rename the columns.

In [321]:
# drop junk columns
title_data_wo_junk = title_data.drop(columns=['Unnamed: 0','region','language','attributes','isOriginalTitle','ordering'])
print(title_data_wo_junk.info())
title_data_wo_junk.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1571 entries, 0 to 1570
Data columns (total 3 columns):
titleId    1571 non-null object
title      1571 non-null object
types      1041 non-null object
dtypes: object(3)
memory usage: 36.9+ KB
None


,titleId,title,types
0,tt0160127,Charlie's Angels,NaN
1,tt0160127,Charlie's Angels: The Movie,working
2,tt0163579,Chill Factor,NaN
3,tt0163651,East Great Falls High,working
4,tt0163651,Great Falls,working


It's looking much better. I'm going to take a closer look at the `types` column to see if it'll help me understand how to handle duplicates.<br>
I've only seen `working` and `NaN` so far as values for this column. I can only guess that `working` refers to a working title of the movie as it's in production. It's probably common for either multiple IMDB pages to exist as a movie is in production or as rumours and speculation arise. Or IMDB could simple append a new record everytime that there's an "official" title change to the movie. I'm going to confirm by checking out the actual IMDB page [https://www.imdb.com/title/tt0160127/](https://www.imdb.com/title/tt0160127/)
<br><br>
As I expected, the `working` very likely refers to a working title, since the "official" title matches the record without `working` as it's type. I'm going to check all of the unique values of the `type` column to see what else there may be.

In [322]:
title_data_wo_junk['types'].unique()

array([nan, 'working', 'imdbDisplay', 'alternative', 'video', 'festival',
       'dvd'], dtype=object)

This is interesting info, but I need to know more before I can make any decisions. I think that `working` type is explained well. But the others are a mystery. I'm not even sure if the other types will apply to duplicate records.<br>
To better understand, I'm going to isolate duplicates in a new dataframe and then check the types again.

In [323]:
# isolate the duplicates to a new dataframe
title_duplicates = title_data_wo_junk[title_data_wo_junk['titleId'].duplicated(keep=False)]
title_duplicates['types'].unique()

array([nan, 'working', 'imdbDisplay', 'alternative', 'video', 'dvd',
       'festival'], dtype=object)

This hasn't helped at all. I need to look at each type to understand it.

In [324]:
title_imdb_display = title_duplicates[title_duplicates['types']=='imdbDisplay']
title_imdb_display.head()

,titleId,title,types
48,tt0210234,Pokémon the Movie 2000,imdbDisplay
95,tt0251736,House of 1000 Corpses,imdbDisplay
128,tt0268126,Adaptation.,imdbDisplay
146,tt0275277,Cowboy Bebop: The Movie,imdbDisplay
197,tt0306734,Le divorce,imdbDisplay


Hmm.. It looks like the titles are somehow out of order because each title is supposed to be a duplicate. Let me single out one title and see if that helps at all. I'm also checking the source to see what the real name is [https://www.imdb.com/title/tt0210234/](https://www.imdb.com/title/tt0210234/)

In [325]:
title_duplicates[title_duplicates['titleId']=='tt0210234']

,titleId,title,types
47,tt0210234,Pokémon: The Power of One,NaN
48,tt0210234,Pokémon the Movie 2000,imdbDisplay
49,tt0210234,Pokémon the Movie 2000 - The Power of One,alternative
50,tt0210234,Pokémon 2,NaN
51,tt0210234,Pokemon: Power of One,NaN


Looks like `imdbDisplay` is what is displayed on IMDB. I'll rank that as highest in priority for now, followed by `NaN` then `working`/`alternative`. I still need to understand `dvd` and `festival`. They seem self explanatory but I must confirm.

In [326]:
title_imdb_display = title_duplicates[title_duplicates['types']=='festival']
title_imdb_display.head()

,titleId,title,types
766,tt0929632,Push,festival
1203,tt2229499,Don Jon's Addiction,festival
1226,tt2334649,Fruitvale,festival
1314,tt3149038,X+Y,festival
1516,tt6803212,Relive,festival


[https://www.imdb.com/title/tt0929632/](https://www.imdb.com/title/tt0929632/)

In [327]:
title_duplicates[title_duplicates['titleId']=='tt0929632']

,titleId,title,types
761,tt0929632,Precious,NaN
762,tt0929632,Precious,NaN
763,tt0929632,Push: Based on the Novel by Sapphire,working
764,tt0929632,Precious: Based on the Novel 'Push' by Sapphire,NaN
765,tt0929632,Precious: Base on Nol by Saf (Based on the Nov...,NaN
766,tt0929632,Push,festival


It looks like the `working` and `festival` titles don't match the "official" title. I think it's safe to rank `festival` with `working`. Some of the `NaN` type titles are incorrect, but I'm comfortable with it. I'm finally tackling `dvd`.

In [328]:
title_imdb_display = title_duplicates[title_duplicates['types']=='dvd']
title_imdb_display.head()

,titleId,title,types
608,tt0480239,Atlas Shrugged: Part One,dvd
785,tt1000774,Sex and the City: The Movie,dvd


[https://www.imdb.com/title/tt1000774/](https://www.imdb.com/title/tt1000774/)

In [329]:
title_duplicates[title_duplicates['titleId']=='tt1000774']

,titleId,title,types
785,tt1000774,Sex and the City: The Movie,dvd
786,tt1000774,Sex and the City,NaN


This is good enough to give `dvd` same priority as the other low ones. Now I just need to sort the dataframe, then dedupe.<br>
Again, the order of priorities for types I've defined is: `imdbDisplay` > `NaN` > `festival` - `working` - `dvd` - `video`<br>
I need to deduplicate based on a prioritized list of strings.<br>
I think the best way is to create a new column with a numeric priority of the type, then sort it in ascending order, then keep the first value while deduping

In [330]:
def prioritize_type(value):
    if value == 'imdbDisplay':
        return(1)
    # testing for NaN in pandas
    elif type(value)==float:
        return(2)
    return(3)
    
title_duplicates_pri = title_duplicates.reset_index()
title_duplicates_pri['pri']=title_duplicates_pri['types'].apply(prioritize_type)
title_duplicates_pri=title_duplicates_pri.sort_values(['titleId','pri'],ascending=True)
title_duplicates_pri.head()

,index,titleId,title,types,pri
0,0,tt0160127,Charlie's Angels,NaN,2
1,1,tt0160127,Charlie's Angels: The Movie,working,3
4,5,tt0163651,American Pie,NaN,2
2,3,tt0163651,East Great Falls High,working,3
3,4,tt0163651,Great Falls,working,3


Success! Everything is sorted the way I need it to. Now I'm going to dedupe and keep the first available record.

In [331]:
title_deduped=title_duplicates_pri.drop_duplicates(subset=['titleId'],keep='first')
title_deduped.head(15)

,index,titleId,title,types,pri
0,0,tt0160127,Charlie's Angels,NaN,2
4,5,tt0163651,American Pie,NaN,2
7,10,tt0166485,Anna and the King,NaN,2
9,14,tt0168786,Antwone Fisher,NaN,2
13,18,tt0172156,Bad Boys II,NaN,2
16,21,tt0175880,Mag·no'li·a,NaN,2
18,24,tt0179098,Moonlight Mile,NaN,2
21,27,tt0183523,M2M,NaN,2
24,31,tt0184858,Reindeer Games,NaN,2
25,39,tt0200469,Bats,NaN,2


Time to combine this with the original non duplicated titles

In [332]:
# isolate the non duplicated titles
title_non_duplicates = title_data_wo_junk[~title_data_wo_junk['titleId'].duplicated(keep=False)]
title_deduped=title_deduped.drop(columns=['index','pri'])
title_ready_for_db=title_non_duplicates.append(title_deduped,ignore_index=True)
print(title_ready_for_db.info())
title_ready_for_db.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 3 columns):
titleId    1001 non-null object
title      1001 non-null object
types      650 non-null object
dtypes: object(3)
memory usage: 23.5+ KB
None


,titleId,title,types
0,tt0163579,Chill Factor,NaN
1,tt0163988,Bringing Out the Dead,imdbDisplay
2,tt0165929,Romeo Must Die,imdbDisplay
3,tt0167456,The Thunderbirds,working
4,tt0168501,The Best Man,imdbDisplay


Great! Our data has come together beautifully! There are exactly the amount of records that we expect. I'm going to drop the type column and rename the titleId column.

In [333]:
title_ready_for_db=title_ready_for_db.drop(columns=['types'])
title_ready_for_db=title_ready_for_db.rename(columns={'titleId':'title_id'})
title_ready_for_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 2 columns):
title_id    1001 non-null object
title       1001 non-null object
dtypes: object(2)
memory usage: 15.7+ KB


## Revistiting the `movie_summary` dataset

In [334]:
print(movie_summary.info())
movie_summary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
titleId            1001 non-null object
dom_total_sales    1001 non-null object
int_total_sales    1001 non-null object
dom_distributer    1000 non-null object
dom_opening        1001 non-null object
budget             1001 non-null object
release_date       1001 non-null object
mpaa_rating        1001 non-null object
dtypes: object(8)
memory usage: 62.6+ KB
None


,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating
0,tt0187393,"$113,330,342","$101,964,000",Sony Pictures ReleasingSee full company inform...,"$22,413,710","$110,000,000","June 28, 2000\n (Domestic)",R
1,tt0283897,"$1,013,272",~na~,United ArtistsSee full company information\n\n,"$64,474",~na~,R,1 hr 54 min
2,tt0475998,"$12,981,269","$266,456",Universal PicturesSee full company information...,"$4,251,320",~na~,G,1 hr 25 min
3,tt0204946,"$68,379,000","$22,070,929",Universal PicturesSee full company information...,"$17,362,105","$11,000,000","August 25, 2000\n (Domestic)",PG-13
4,tt1549920,"$12,050,299","$36,280,458",LionsgateSee full company information\n\n,"$6,281,433","$45,000,000","January 17, 2013\n (APAC, EMEA)",R


The actual number of records matches the expected number. But I see that there are some unhandled missing data. I can fix this by re-reading the data in and specifying what missing data looks like.

In [335]:
movie_summary = pd.read_csv('movie_summary.csv',na_values='~na~')
print(movie_summary.info())
movie_summary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
titleId            1001 non-null object
dom_total_sales    1001 non-null object
int_total_sales    933 non-null object
dom_distributer    1000 non-null object
dom_opening        992 non-null object
budget             561 non-null object
release_date       1001 non-null object
mpaa_rating        1001 non-null object
dtypes: object(8)
memory usage: 62.6+ KB
None


,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating
0,tt0187393,"$113,330,342","$101,964,000",Sony Pictures ReleasingSee full company inform...,"$22,413,710","$110,000,000","June 28, 2000\n (Domestic)",R
1,tt0283897,"$1,013,272",NaN,United ArtistsSee full company information\n\n,"$64,474",NaN,R,1 hr 54 min
2,tt0475998,"$12,981,269","$266,456",Universal PicturesSee full company information...,"$4,251,320",NaN,G,1 hr 25 min
3,tt0204946,"$68,379,000","$22,070,929",Universal PicturesSee full company information...,"$17,362,105","$11,000,000","August 25, 2000\n (Domestic)",PG-13
4,tt1549920,"$12,050,299","$36,280,458",LionsgateSee full company information\n\n,"$6,281,433","$45,000,000","January 17, 2013\n (APAC, EMEA)",R


This is already looking better. I'm going to rename the `titleId` column once again and then ideally convert the currency to float. I see that some `mpaa_rating` values ended up in the `release_date` column. Unfortunately, this lowers my confidence in this dataset. It's going to be tricky to think of a clever solution to fix this on the fly. I'd hate to throw these columns away though. Another thing I'm worried about is that something similar may be happening with columns such as `dom_opening` and `budget`. It will be tricky to detect where values in those columns could be flipped around. Fortunately, I can infer the `release_date` value from the day 1 sale figure from the `domestic_sales` table column `date`, so I can toss that column out without losing much.
<br><br><br>
In the meanwhile, I'm going to build a function to try and detect non-numeric values in any currency column

In [336]:
def is_currency(value):
    numeric_value = value.replace('$','').replace(',','')
    result = re.match(r'[^0-9]',numeric_value)
    if result:
        return(True)
    return(False)

In [337]:
movie_dom = movie_summary
movie_dom['dom_currency']=movie_dom['dom_total_sales'].apply(is_currency)
movie_summary[movie_dom['dom_currency']]

,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating,dom_currency


That looks good. I'm going to apply the same method to the other currency types and determine if it's safe to cast them to numeric

In [338]:
movie_dom['int_currency']=movie_dom['int_total_sales'].apply(is_currency)
movie_dom['dom_op']=movie_dom['dom_opening'].apply(is_currency)
movie_dom['bud']=movie_dom['budget'].apply(is_currency)
movie_summary[movie_dom['int_currency']]

AttributeError: 'float' object has no attribute 'replace'

Looks like it's unsafe to replace strings in the `int_total_sales` column. It's because there are null values in there. I'm going to replace the null values with `12345` so that I can still tell that those were initially null. I'd typically store it as `-1` but that'll get caught by the non-number regex.

In [339]:
# create a new dataframe to store this still
movie_filled_nas = movie_summary
movie_filled_nas['int_total_sales']=movie_filled_nas['int_total_sales'].fillna('12345')
movie_filled_nas['dom_opening']=movie_filled_nas['dom_opening'].fillna('12345')
movie_filled_nas['budget']=movie_filled_nas['budget'].fillna('12345')
movie_filled_nas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 9 columns):
titleId            1001 non-null object
dom_total_sales    1001 non-null object
int_total_sales    1001 non-null object
dom_distributer    1000 non-null object
dom_opening        1001 non-null object
budget             1001 non-null object
release_date       1001 non-null object
mpaa_rating        1001 non-null object
dom_currency       1001 non-null bool
dtypes: bool(1), object(8)
memory usage: 63.6+ KB


Looking good, I'm going to proceed.

In [340]:
movie_dom = movie_filled_nas
movie_dom['int_currency']=movie_dom['int_total_sales'].apply(is_currency)
movie_dom['dom_op']=movie_dom['dom_opening'].apply(is_currency)
movie_dom['bud']=movie_dom['budget'].apply(is_currency)
movie_summary[movie_dom['int_currency']]

,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating,dom_currency,int_currency,dom_op,bud


In [341]:
movie_summary[movie_dom['dom_op']]

,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating,dom_currency,int_currency,dom_op,bud


In [342]:
movie_summary[movie_dom['bud']]

,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating,dom_currency,int_currency,dom_op,bud


Great, this tells me that these column are safe to cast to float and character values did not slip in accidentally.

In [343]:
movie_cast_to_num = movie_filled_nas
movie_cast_to_num['int_total_sales']=movie_cast_to_num['int_total_sales'].str.replace('$','')
movie_cast_to_num['int_total_sales']=pd.to_numeric(movie_cast_to_num['int_total_sales'].str.replace(',',''))
print(movie_cast_to_num.info())
movie_cast_to_num.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 12 columns):
titleId            1001 non-null object
dom_total_sales    1001 non-null object
int_total_sales    1001 non-null int64
dom_distributer    1000 non-null object
dom_opening        1001 non-null object
budget             1001 non-null object
release_date       1001 non-null object
mpaa_rating        1001 non-null object
dom_currency       1001 non-null bool
int_currency       1001 non-null bool
dom_op             1001 non-null bool
bud                1001 non-null bool
dtypes: bool(4), int64(1), object(7)
memory usage: 66.6+ KB
None


,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating,dom_currency,int_currency,dom_op,bud
0,tt0187393,"$113,330,342",101964000,Sony Pictures ReleasingSee full company inform...,"$22,413,710","$110,000,000","June 28, 2000\n (Domestic)",R,False,False,False,False
1,tt0283897,"$1,013,272",12345,United ArtistsSee full company information\n\n,"$64,474",12345,R,1 hr 54 min,False,False,False,False
2,tt0475998,"$12,981,269",266456,Universal PicturesSee full company information...,"$4,251,320",12345,G,1 hr 25 min,False,False,False,False
3,tt0204946,"$68,379,000",22070929,Universal PicturesSee full company information...,"$17,362,105","$11,000,000","August 25, 2000\n (Domestic)",PG-13,False,False,False,False
4,tt1549920,"$12,050,299",36280458,LionsgateSee full company information\n\n,"$6,281,433","$45,000,000","January 17, 2013\n (APAC, EMEA)",R,False,False,False,False


I just printed this out to confirm that my method worked. I'm going to apply it to the others.

In [344]:
movie_cast_to_num['dom_total_sales']=movie_cast_to_num['dom_total_sales'].str.replace('$','')
movie_cast_to_num['dom_total_sales']=pd.to_numeric(movie_cast_to_num['dom_total_sales'].str.replace(',',''))
movie_cast_to_num['dom_opening']=movie_cast_to_num['dom_opening'].str.replace('$','')
movie_cast_to_num['dom_opening']=pd.to_numeric(movie_cast_to_num['dom_opening'].str.replace(',',''))
movie_cast_to_num['budget']=movie_cast_to_num['budget'].str.replace('$','')
movie_cast_to_num['budget']=pd.to_numeric(movie_cast_to_num['budget'].str.replace(',',''))
print(movie_cast_to_num.info())
movie_cast_to_num.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 12 columns):
titleId            1001 non-null object
dom_total_sales    1001 non-null int64
int_total_sales    1001 non-null int64
dom_distributer    1000 non-null object
dom_opening        1001 non-null int64
budget             1001 non-null int64
release_date       1001 non-null object
mpaa_rating        1001 non-null object
dom_currency       1001 non-null bool
int_currency       1001 non-null bool
dom_op             1001 non-null bool
bud                1001 non-null bool
dtypes: bool(4), int64(4), object(4)
memory usage: 66.6+ KB
None


,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget,release_date,mpaa_rating,dom_currency,int_currency,dom_op,bud
0,tt0187393,113330342,101964000,Sony Pictures ReleasingSee full company inform...,22413710,110000000,"June 28, 2000\n (Domestic)",R,False,False,False,False
1,tt0283897,1013272,12345,United ArtistsSee full company information\n\n,64474,12345,R,1 hr 54 min,False,False,False,False
2,tt0475998,12981269,266456,Universal PicturesSee full company information...,4251320,12345,G,1 hr 25 min,False,False,False,False
3,tt0204946,68379000,22070929,Universal PicturesSee full company information...,17362105,11000000,"August 25, 2000\n (Domestic)",PG-13,False,False,False,False
4,tt1549920,12050299,36280458,LionsgateSee full company information\n\n,6281433,45000000,"January 17, 2013\n (APAC, EMEA)",R,False,False,False,False


Now that the data is in the correct format, I'm going to re-replace the missing values with `-1` and then drop unneccesary columns

In [345]:
# drop unneccesary columns
movies_fix_na = movie_cast_to_num.drop(columns=['release_date','mpaa_rating','dom_currency','int_currency','dom_op','bud'])

# re-replace missing values
movies_fix_na.loc[movies_fix_na['int_total_sales']==12345,'int_total_sales']=-1
movies_fix_na.loc[movies_fix_na['dom_opening']==12345,'dom_opening']=-1
movies_fix_na.loc[movies_fix_na['budget']==12345,'budget']=-1

movies_fix_na.head()

,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget
0,tt0187393,113330342,101964000,Sony Pictures ReleasingSee full company inform...,22413710,110000000
1,tt0283897,1013272,-1,United ArtistsSee full company information\n\n,64474,-1
2,tt0475998,12981269,266456,Universal PicturesSee full company information...,4251320,-1
3,tt0204946,68379000,22070929,Universal PicturesSee full company information...,17362105,11000000
4,tt1549920,12050299,36280458,LionsgateSee full company information\n\n,6281433,45000000


Lastly, I think I have a quick solution to fix the `dom_distributer` column. That column is the domestic distributer for the movie and it looks like "See full..." is always concatenated to the name due to the way I scraped the data. I think I can split the value based on "See full" and just take the first half.

In [346]:
movie_ready_for_db = movies_fix_na
movie_ready_for_db['dom_distributer']=movie_ready_for_db['dom_distributer'].str.split('See full').str[0]
movie_ready_for_db.head()

,titleId,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget
0,tt0187393,113330342,101964000,Sony Pictures Releasing,22413710,110000000
1,tt0283897,1013272,-1,United Artists,64474,-1
2,tt0475998,12981269,266456,Universal Pictures,4251320,-1
3,tt0204946,68379000,22070929,Universal Pictures,17362105,11000000
4,tt1549920,12050299,36280458,Lionsgate,6281433,45000000


Beautiful, I'm going to take a peek at the unique values to ensure nothing messed up with the distributers. I'm also going to rename the `titleId` column once again

In [347]:
movie_ready_for_db=movie_ready_for_db.rename(columns={'titleId':'title_id'})
movie_ready_for_db['dom_distributer'].unique()

array(['Sony Pictures Releasing', 'United Artists', 'Universal Pictures',
       'Lionsgate', 'Twentieth Century Fox', 'UTV Motion Pictures',
       'RADiUS-TWC', 'DreamWorks', 'Walt Disney Studios Motion Pictures',
       'P & A Releasing', 'Paramount Pictures', 'Warner Bros.',
       'IFC Films', 'Anchor Bay Films', 'CBS Films', 'Paramount Classics',
       'Cohen Media Group', 'Overture Films', 'Miramax', 'THINKFilm',
       'BH Tilt', 'Summit Entertainment', 'Fox Searchlight Pictures',
       'Roadside Attractions', 'Revolution Studios', 'Entertainment One',
       'Slowhand Cinema Releasing', 'Open Road Films (II)',
       'Vertical Entertainment', 'Dimension Films', 'Indomina Media',
       'Rocky Mountain Pictures', 'TriStar Pictures',
       'The Weinstein Company', 'FIP', 'GKIDS',
       'Fun Academy Motion Pictures', 'Sony Pictures Classics',
       'STX Entertainment', 'The Cinema Guild', 'Screen Gems',
       'Paramount Vantage', 'FilmDistrict', 'Freestyle Releasing',
     

It looks great. I'm ready to call this done.

## Revisiting box office numbers

In [348]:
print(domestic_sales.info())
domestic_sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64905 entries, 0 to 64904
Data columns (total 4 columns):
Date       64905 non-null object
Daily      64905 non-null object
Day        64905 non-null object
titleId    64905 non-null object
dtypes: object(4)
memory usage: 2.0+ MB
None


,Date,Daily,Day,titleId
0,"Jun 28, 2000","$5,015,267",1,tt0187393
1,"Jun 29, 2000","$4,307,385",2,tt0187393
2,"Jun 30, 2000","$6,860,000",3,tt0187393
3,"Jul 1, 2000","$8,560,000",4,tt0187393
4,"Jul 2, 2000","$6,990,000",5,tt0187393


By far the cleanest of the three datasets, the only thing I see that needs to be done is to rename the columns, cast the columns to their appropriate datatype, and I think I'm going to create a column for day of the week.<br><br>
I'm going to start by renaming the columns

In [349]:
dom_sales_renamed = domestic_sales.rename(columns={'titleId':'title_id',
                                                  'Date':'date',
                                                  'Daily':'daily',
                                                  'Day':'day'})
dom_sales_renamed=dom_sales_renamed[['title_id','date','daily','day']]
dom_sales_renamed.head()

,title_id,date,daily,day
0,tt0187393,"Jun 28, 2000","$5,015,267",1
1,tt0187393,"Jun 29, 2000","$4,307,385",2
2,tt0187393,"Jun 30, 2000","$6,860,000",3
3,tt0187393,"Jul 1, 2000","$8,560,000",4
4,tt0187393,"Jul 2, 2000","$6,990,000",5


I'm a fan of this new ordering. I'm confident with the method I scraped this with, as well as the method for casting currency to numeric earlier, so I'm going to directly cast `daily` to numeric.

In [350]:
dom_sales_renamed['daily']=dom_sales_renamed['daily'].str.replace('$','')
dom_sales_renamed['daily']=pd.to_numeric(dom_sales_renamed['daily'].str.replace(',',''))
print(dom_sales_renamed.info())
dom_sales_renamed.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64905 entries, 0 to 64904
Data columns (total 4 columns):
title_id    64905 non-null object
date        64905 non-null object
daily       64905 non-null int64
day         64905 non-null object
dtypes: int64(1), object(3)
memory usage: 2.0+ MB
None


,title_id,date,daily,day
0,tt0187393,"Jun 28, 2000",5015267,1
1,tt0187393,"Jun 29, 2000",4307385,2
2,tt0187393,"Jun 30, 2000",6860000,3
3,tt0187393,"Jul 1, 2000",8560000,4
4,tt0187393,"Jul 2, 2000",6990000,5


I'm going to cast `date` to a `datetime` object. I think this is a bit unsafe without exploring the data a bit more first, but I want to see how effective the native `pandas` method is.

In [351]:
dom_sales_renamed['date']=pd.to_datetime(dom_sales_renamed['date'])
print(dom_sales_renamed.info())
dom_sales_renamed.head()

ValueError: ('Unknown string format:', 'Jul 4, 2000Independence Day')

Dang.. I forgot that major holidays got appended to the date string when I scraped this. I'm going to have to build a custom method.

In [352]:
def clean_date_string(value):
    mon_day = re.compile(r'(19|20)\d\d').split(value)[0]
    result = re.search('(19|20)\d\d',value)
    return mon_day + result.group(0)

In [353]:
dom_new_date = dom_sales_renamed
dom_new_date['date']=dom_new_date['date'].apply(clean_date_string)
dom_new_date['date']=pd.to_datetime(dom_new_date['date'])
print(dom_new_date.info())
dom_new_date.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64905 entries, 0 to 64904
Data columns (total 4 columns):
title_id    64905 non-null object
date        64905 non-null datetime64[ns]
daily       64905 non-null int64
day         64905 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.0+ MB
None


,title_id,date,daily,day
0,tt0187393,2000-06-28,5015267,1
1,tt0187393,2000-06-29,4307385,2
2,tt0187393,2000-06-30,6860000,3
3,tt0187393,2000-07-01,8560000,4
4,tt0187393,2000-07-02,6990000,5


It looks like that wasn't very efficient but it worked! The last thing I'm going to do is create a new column for day of the week.<br>
The `pandas.Series` object has a method called `dt.dayofweek` which will convert a date to the day of week, with Monday=0 and Sunday=6

In [354]:
dom_ready_for_db = dom_new_date
dom_ready_for_db['day_of_week']=dom_ready_for_db['date'].dt.dayofweek
print(dom_ready_for_db.info())
dom_ready_for_db.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64905 entries, 0 to 64904
Data columns (total 5 columns):
title_id       64905 non-null object
date           64905 non-null datetime64[ns]
daily          64905 non-null int64
day            64905 non-null object
day_of_week    64905 non-null int64
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 2.5+ MB
None


,title_id,date,daily,day,day_of_week
0,tt0187393,2000-06-28,5015267,1,2
1,tt0187393,2000-06-29,4307385,2,3
2,tt0187393,2000-06-30,6860000,3,4
3,tt0187393,2000-07-01,8560000,4,5
4,tt0187393,2000-07-02,6990000,5,6


Not done yet, I just realized that day is of type `object` and it should be numeric. I'm going to take a look.

In [357]:
dom_ready_for_db[~dom_ready_for_db['day'].str.isnumeric()]

,title_id,date,daily,day,day_of_week
691,tt0449088,2007-05-24,13240044,-,3
18399,tt2306745,2013-01-31,120514,-,3
19720,tt3086442,2015-09-10,3000,-,3
29626,tt1477076,2010-10-28,1700000,-,3
33618,tt0800320,2010-04-01,2655005,-,3
35248,tt0796366,2009-05-07,4000000,-,3
42162,tt1399103,2011-06-28,5500000,-,1
50032,tt0295178,2002-07-25,3553719,-,3
55932,tt1438176,2011-08-18,400000,-,3
58582,tt1702443,2011-02-09,742547,-,2


I took a look at the source for these. These records seem to appear a day before the movie's theatrical release. Maybe a special screening or something. I'm comfortable setting these records to 0.

In [358]:
dom_ready_for_db_final = dom_ready_for_db
dom_ready_for_db_final.loc[~dom_ready_for_db_final['day'].str.isnumeric(),'day']='0'
dom_ready_for_db_final['day']=pd.to_numeric(dom_ready_for_db_final['day'])
dom_ready_for_db_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64905 entries, 0 to 64904
Data columns (total 5 columns):
title_id       64905 non-null object
date           64905 non-null datetime64[ns]
daily          64905 non-null int64
day            64905 non-null int64
day_of_week    64905 non-null int64
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 2.5+ MB


## Prepare to write datasets to files
I'm pretty content with the way that these datasets turned out. The only other thing I want to do is merge the title info and the movie summary. Both files exist in the exact same format and they are lightweight to begin with. I don't see a need to keep them separated.

In [359]:
overall_title_ready_for_db = title_ready_for_db.merge(movie_ready_for_db,on='title_id')
print(overall_title_ready_for_db.info())
overall_title_ready_for_db.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 7 columns):
title_id           1001 non-null object
title              1001 non-null object
dom_total_sales    1001 non-null int64
int_total_sales    1001 non-null int64
dom_distributer    1000 non-null object
dom_opening        1001 non-null int64
budget             1001 non-null int64
dtypes: int64(4), object(3)
memory usage: 62.6+ KB
None


,title_id,title,dom_total_sales,int_total_sales,dom_distributer,dom_opening,budget
0,tt0163579,Chill Factor,11263966,524710,Warner Bros.,4529231,70000000
1,tt0163988,Bringing Out the Dead,16797191,-1,Paramount Pictures,6193052,55000000
2,tt0165929,Romeo Must Die,55973336,35063424,Warner Bros.,18014503,25000000
3,tt0167456,The Thunderbirds,6880917,21402720,Universal Pictures,2766810,57000000
4,tt0168501,The Best Man,34102780,471000,Universal Pictures,9031660,9000000


Nice! Now the datasets are ready to write out.

In [360]:
overall_title_ready_for_db.to_csv('overall_title_for_db.csv',index=False)
dom_ready_for_db_final.to_csv('domestic_sales_for_db.csv',index=False)